In [1]:
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load Boston dataset
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep=r"\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# Convert data to DataFrame and assign X, y
X = pd.DataFrame(data, columns=[
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 
    'TAX', 'PTRATIO', 'B', 'LSTAT'
])
y = pd.Series(target, name='MEDV')

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

def get_best_subset(X, y, k):
    """ Finds the best subset of k predictors based on training RSS. """
    best_model = None
    best_rss = np.inf
    for subset in itertools.combinations(X.columns, k):
        X_subset = X[list(subset)]
        X_subset = sm.add_constant(X_subset)  # add constant for intercept
        model = sm.OLS(y, X_subset).fit()
        rss = ((model.predict(X_subset) - y) ** 2).sum()
        if rss < best_rss:
            best_rss = rss
            best_model = model
    return best_model

# Best subset selection
best_subset_models = {}
for k in range(1, 6):  # Limiting to models with up to 5 predictors
    best_subset_models[k] = get_best_subset(X_train, y_train, k)

# Forward Stepwise Selection
def forward_stepwise(X, y, max_features):
    remaining_predictors = list(X.columns)
    selected_predictors = []
    models = []
    for _ in range(max_features):
        best_rss = np.inf
        best_predictor = None
        for predictor in remaining_predictors:
            model_predictors = selected_predictors + [predictor]
            X_model = X[model_predictors]
            X_model = sm.add_constant(X_model)
            model = sm.OLS(y, X_model).fit()
            rss = ((model.predict(X_model) - y) ** 2).sum()
            if rss < best_rss:
                best_rss = rss
                best_predictor = predictor
        selected_predictors.append(best_predictor)
        remaining_predictors.remove(best_predictor)
        models.append(sm.OLS(y, sm.add_constant(X[selected_predictors])).fit())
    return models

forward_stepwise_models = forward_stepwise(X_train, y_train, max_features=5)

# Backward Stepwise Selection
def backward_stepwise(X, y):
    selected_predictors = list(X.columns)
    models = [sm.OLS(y, sm.add_constant(X[selected_predictors])).fit()]
    while len(selected_predictors) > 1:
        best_rss = np.inf
        worst_predictor = None
        for predictor in selected_predictors:
            model_predictors = [p for p in selected_predictors if p != predictor]
            X_model = X[model_predictors]
            X_model = sm.add_constant(X_model)
            model = sm.OLS(y, X_model).fit()
            rss = ((model.predict(X_model) - y) ** 2).sum()
            if rss < best_rss:
                best_rss = rss
                worst_predictor = predictor
        selected_predictors.remove(worst_predictor)
        models.append(sm.OLS(y, sm.add_constant(X[selected_predictors])).fit())
    return models

backward_stepwise_models = backward_stepwise(X_train, y_train)

# Testing RSS Comparison
def calculate_test_rss(models, X_test, y_test):
    rss_list = []
    for model in models:
        predictors = model.model.exog_names
        X_test_subset = X_test[predictors[1:]]  # exclude constant term
        X_test_subset = sm.add_constant(X_test_subset)
        y_pred = model.predict(X_test_subset)
        rss = ((y_pred - y_test) ** 2).sum()
        rss_list.append(rss)
    return rss_list

# Calculate RSS for test data
best_subset_rss = calculate_test_rss(best_subset_models.values(), X_test, y_test)
forward_stepwise_rss = calculate_test_rss(forward_stepwise_models, X_test, y_test)
backward_stepwise_rss = calculate_test_rss(backward_stepwise_models, X_test, y_test)

# Print results
print("Best Subset Selection Test RSS:", best_subset_rss)
print("Forward Stepwise Selection Test RSS:", forward_stepwise_rss)
print("Backward Stepwise Selection Test RSS:", backward_stepwise_rss)



Best Subset Selection Test RSS: [6478.277007674038, 4398.04268384516, 3860.938230651976, 3607.0915411657306, 3406.9094458522673]
Forward Stepwise Selection Test RSS: [6478.277007674038, 4398.042683845158, 3860.938230651973, 3607.091541165726, 3406.9094458522527]
Backward Stepwise Selection Test RSS: [3014.361198153569, 3013.5408061796443, 2988.8817999575567, 3150.5262991775944, 3333.6484843139674, 3437.634333492885, 3377.152670481532, 3325.513189105136, 3406.9094458522673, 3607.0915411657306, 3860.938230651976, 4398.04268384516, 6478.277007674038]


In [2]:
import pandas as pd
import statsmodels.api as sm

# Load the Auto dataset
file_path = 'C:/Users/arman/Downloads/ALL+CSV+FILES+-+2nd+Edition+-+corrected/ALL CSV FILES - 2nd Edition/Auto.csv'
auto_data = pd.read_csv(file_path)

# Convert horsepower to numeric, coerce errors to NaN, and drop rows with NaN values
auto_data['horsepower'] = pd.to_numeric(auto_data['horsepower'], errors='coerce')
auto_data.dropna(subset=['horsepower', 'mpg'], inplace=True)

# Define predictor (X) and response (y)
X = sm.add_constant(auto_data['horsepower'])  # Adding a constant for intercept
y = auto_data['mpg']

# Perform simple linear regression using sm.OLS
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

# Calculate the predicted mpg for horsepower = 98
horsepower_98 = pd.DataFrame({'const': 1, 'horsepower': [98]})
predicted_mpg_98 = model.get_prediction(horsepower_98)

# Get the confidence and prediction intervals at 95% confidence level
prediction_summary_98 = predicted_mpg_98.summary_frame(alpha=0.05)
print(prediction_summary_98[['mean', 'mean_ci_lower', 'mean_ci_upper', 'obs_ci_lower', 'obs_ci_upper']])



                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Sat, 26 Oct 2024   Prob (F-statistic):           7.03e-81
Time:                        23:00:48   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9359      0.717     55.660      0.0

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error

# Load the data
train_data = pd.read_csv('C:/Users/arman/Downloads/playground-series-s4e4/train.csv')
test_data = pd.read_csv('C:/Users/arman/Downloads/playground-series-s4e4/test.csv')

# Data Preparation
# Drop duplicate columns in train and test data
train_data = train_data.drop(columns=['Whole weight.1'])
test_data = test_data.drop(columns=['Whole weight.1'])

# Encode categorical feature 'Sex'
le = LabelEncoder()
train_data['Sex'] = le.fit_transform(train_data['Sex'])
test_data['Sex'] = le.transform(test_data['Sex'])

# Separate features and target in training data
X = train_data.drop(columns=['id', 'Rings'])
y = train_data['Rings']

# Split data for model validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Model 1: Ridge Regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_val_ridge = ridge_model.predict(X_val)
rmsle_ridge = np.sqrt(mean_squared_log_error(y_val, np.maximum(0, y_pred_val_ridge)))

print("RMSLE (Ridge Model):", rmsle_ridge)

# Model 2: Principal Component Regression (PCR)
pca_pipeline = Pipeline([
    ('pca', PCA(n_components=5)),  # Using 5 principal components
    ('linear_regression', LinearRegression())
])

# Fit and predict with PCA model
pca_pipeline.fit(X_train, y_train)
y_pred_val_pca = pca_pipeline.predict(X_val)
rmsle_pca = np.sqrt(mean_squared_log_error(y_val, np.maximum(0, y_pred_val_pca)))

print("RMSLE (PCR Model):", rmsle_pca)

# Predictions on Test Data
ridge_predictions = ridge_model.predict(test_data.drop(columns=['id']))
pca_predictions = pca_pipeline.predict(test_data.drop(columns=['id']))

# Create submission files
submission_ridge = pd.DataFrame({'id': test_data['id'], 'Rings': np.maximum(0, ridge_predictions).round().astype(int)})
submission_pca = pd.DataFrame({'id': test_data['id'], 'Rings': np.maximum(0, pca_predictions).round().astype(int)})

# Save submission files
submission_ridge.to_csv('C:/Users/arman/Downloads/ridge_submission.csv', index=False)
submission_pca.to_csv('C:/Users/arman/Downloads/pca_submission.csv', index=False)


RMSLE (Ridge Model): 0.17256617684408151
RMSLE (PCR Model): 0.17457462367892795
